# Phân loại tập văn bản 20newsgroup với mạng ANN 

## Các công việc cần thực hiện:

- Xử lý tập dữ liệu 20newsgroup, trong đó:
  + Phân tách dữ liệu thành 2 tập để phục vụ do việc học và kiểm thử
  + Lưu dữ liệu vào các danh sách để có thể xử lý được (20newsgroups_train.data và 20newsgroups_test.data)
  + Tạo danh mục các từ 
- Xây dựng mạng neural:
  + Sử dụng thư viện pytorch
  + Mạng neural feed-forward gồm 2 tầng
- Training mô hình:
  + Thực hiện training 15 lần
  + Tập train được chia thành các gói ứng với các đầu vào (input)
  + Sử dụng hàm loss: CrossEntropy, hàm tối ưu : Adaptive Moment Estimation (Adam)

In [3]:
import os
import shutil
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.base import Bunch
import random
from sklearn.feature_extraction.text import CountVectorizer
import re

In [57]:
#Tạo tập train và tập test
os.makedirs('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_train')
os.makedirs('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_test')

In [58]:
#Tạo các thư mục con trong tập train và test
for i in os.listdir('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20news-18828'):    
    sub_directory_train = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_train' + '\\' + i
    sub_directory_test = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_test' + '\\' + i
    os.makedirs(sub_directory_train)
    os.makedirs(sub_directory_test)

In [59]:
#copy các file sang folder train và test
for i in os.listdir('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20news-18828'):
    sub_directory = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20news-18828' + '\\' + i
    folder_length = len(os.listdir(sub_directory))
    
    train_length = round(folder_length * 0.6)   # Độ dài tập train (60%)
    test_length = folder_length - train_length  # Độ dài tập test  (40%)
    
    sub_directory_train = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_train' + '\\' + i
    sub_directory_test = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_test' + '\\' + i
    
    for news_file in os.listdir(sub_directory)[:train_length]:
        news_file_src = sub_directory + '\\' + news_file
        shutil.copy(news_file_src, sub_directory_train)
        
    for news_file in os.listdir(sub_directory)[train_length:]:
        news_file_src = sub_directory + '\\' + news_file
        shutil.copy(news_file_src, sub_directory_test)

In [60]:
#Tạo file label cho tập train
f = open("E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\train_label.txt","w+")

label = 0
for i in os.listdir('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_train'):
    sub_directory_train = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_train' + '\\' + i
    for news_file in os.listdir(sub_directory_train):
        f.write("%d\n" %label)
    label += 1

f.close()

In [61]:
#Tạo file label cho tập test
f = open("E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\test_label.txt","w+")

label = 0
for i in os.listdir('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_test'):
    sub_directory_test = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_test' + '\\' + i
    for news_file in os.listdir(sub_directory_test):
        f.write("%d\n" %label)
    label += 1

f.close()

In [4]:
newsgroups_train = Bunch()
newsgroups_train.data = []
newsgroups_train.target = []

newsgroups_test = Bunch()
newsgroups_test.data = []
newsgroups_test.target = []

In [5]:
# đưa dữ diệu từ file tập train vào newsgroups_train.data
for i in os.listdir('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_train'):
    sub_directory_train = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_train' + '\\' + i
    for news_file in os.listdir(sub_directory_train):
        news_file_path = sub_directory_train + '\\' + news_file
        with open(news_file_path) as f:
            news = f.read()
            newsgroups_train.data.append(news)

In [6]:
len(newsgroups_train.data)

11296

In [7]:
# đưa dữ diệu từ file tập test vào newsgroups_test.data
for i in os.listdir('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_test'):
    sub_directory_test = 'E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\20newsgroup_test' + '\\' + i
    for news_file in os.listdir(sub_directory_test):
        news_file_path = sub_directory_test + '\\' + news_file
        with open(news_file_path) as f:
            news = f.read()
            newsgroups_test.data.append(news)

In [8]:
len(newsgroups_test.data)

7532

In [9]:
# đưa các nhãn lớp (chủ đề) của các file train vào newsgroups_train.target

with open('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\train_label.txt') as f :
    all_the_lines = f.readlines()
    for i in all_the_lines: 
        i = int(i)
        newsgroups_train.target.append(i)

In [10]:
print(len(newsgroups_train.target))

11296


In [11]:
# đưa các nhãn lớp (chủ đề) của các file test vào newsgroups_test.target

with open('E:\\Project 1\\20_newsgroups_exercise\\20_newsgroup_train_test\\test_label.txt') as f :
    all_the_lines = f.readlines()
    for i in all_the_lines:         
        i = int(i)
        newsgroups_test.target.append(i)

In [12]:
print(len(newsgroups_test.target))

7532


In [13]:
#Làm xáo trộn tập train
mapIndexPosition = list(zip(newsgroups_train.data, newsgroups_train.target))
random.shuffle(mapIndexPosition)
newsgroups_train.data, newsgroups_train.target = zip(*mapIndexPosition)

In [14]:
#Làm xáo trộn tập test
mapIndexPosition = list(zip(newsgroups_test.data, newsgroups_test.target))
random.shuffle(mapIndexPosition)
newsgroups_test.data, newsgroups_test.target = zip(*mapIndexPosition)

In [15]:
# Hàm tạo danh sách các từ
def get_word_index(df1, df2):
    vectorizer = CountVectorizer()
    vocab = {}
    texts = ''
    for text in df1.data:
        texts = texts + text
    for text in df2.data:
        texts = texts + text
        
    vectorizer.fit([texts])
    vocab.update(vectorizer.vocabulary_)
    return vocab
wordindex = get_word_index(newsgroups_train, newsgroups_test)
total_words = len(wordindex)

total_words

170781

In [16]:
wordindex

{'from': 74959,
 'eechen': 66729,
 'leland': 98987,
 'stanford': 144385,
 'edu': 66676,
 'emery': 67796,
 'ethan': 69432,
 'chen': 52299,
 'subject': 145756,
 're': 130962,
 'let': 99155,
 'talk': 148337,
 'phillies': 122938,
 'article': 39805,
 'as': 39878,
 'follows': 73974,
 'bml2': 45561,
 'ns1': 115859,
 'cc': 51147,
 'lehigh': 98946,
 'brian': 46768,
 'michael': 107518,
 'lucy': 101472,
 'date': 59732,
 '15': 6049,
 'apr': 39045,
 '93': 29347,
 '06': 1479,
 '29': 13735,
 '05': 1332,
 'gmt': 78318,
 'organization': 118753,
 'university': 155808,
 'lines': 99947,
 'in': 87625,
 'uflkll_00vpcekw15e': 154526,
 'andrew': 38001,
 'cmu': 54041,
 'al1x': 36756,
 'amit': 37622,
 'likhy': 99805,
 'ani': 38113,
 'writes': 163978,
 'excerpts': 70117,
 'netnews': 113979,
 'rec': 131239,
 'sport': 143556,
 'baseball': 42858,
 'u96_msopher': 154073,
 'vaxc': 158192,
 'stevens': 144876,
 '963': 29997,
 'like': 99787,
 'this': 150195,
 'oh': 117610,
 'well': 162017,
 'how': 84421,
 'do': 63775,
 

In [17]:
#Hàm tạo các gói(batch) để làm đầu vào(input) khi training cũng như khi test
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.data[i*batch_size:i*batch_size+batch_size]
    categories = df.target[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        layer = np.zeros(total_words,dtype=float)
        processed_text = text.lower()     
        processed_text = re.sub('[^a-z]+', " ", processed_text)
        
        for word in processed_text.split(' '):
            if word in wordindex:
                layer[wordindex[word]] += 1     # Có thể coi layer ở đây là một feature vector của văn bản với số chiều là tổng
                                                # số từ trong từ điển. Mỗi phần tử trong layer đại diện cho số từ tương ứng  
                                                # xuất hiện trong văn bản đó.
        batches.append(layer)

    for category in categories:
        index_y = -1
        for j in range(20):
            if category == j:
                index_y = j
                break
    
        results.append(index_y)


    return np.array(batches),np.array(results)

In [18]:
# Các tham số
learning_rate = 0.01
num_epochs = 15        #Số lần học
batch_size = 150       #Kích thước các gói 
display_step = 1

# Tham số của mạng(network)
hidden_size = 100        # Kích thước của các tầng trong mạng
input_size = total_words # Tổng số lượng các từ
num_classes = 20         # Tổng số nhãn lớp

In [19]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

In [20]:
class OurNet(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
        super(OurNet, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

     def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [21]:
net = OurNet(input_size, hidden_size, num_classes)

In [20]:
# Training num_epochs = 15
# Hàm Loss và Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    #  Lặp lại qua từng gói
    for i in range(total_batch + 1):
        batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()            # Đưa các giá trị gradient về 0
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()                  # Tính toán các gradient  
        optimizer.step()                 # Cập nhật các tham số

        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.data))

Epoch [1/15], Step [4/75], Loss: 2.8683
Epoch [1/15], Step [8/75], Loss: 2.4600
Epoch [1/15], Step [12/75], Loss: 1.8625
Epoch [1/15], Step [16/75], Loss: 1.6037
Epoch [1/15], Step [20/75], Loss: 1.1232
Epoch [1/15], Step [24/75], Loss: 1.0060
Epoch [1/15], Step [28/75], Loss: 0.8692
Epoch [1/15], Step [32/75], Loss: 1.4223
Epoch [1/15], Step [36/75], Loss: 0.6691
Epoch [1/15], Step [40/75], Loss: 0.6539
Epoch [1/15], Step [44/75], Loss: 0.9981
Epoch [1/15], Step [48/75], Loss: 0.6884
Epoch [1/15], Step [52/75], Loss: 0.6477
Epoch [1/15], Step [56/75], Loss: 0.4220
Epoch [1/15], Step [60/75], Loss: 0.5173
Epoch [1/15], Step [64/75], Loss: 0.4559
Epoch [1/15], Step [68/75], Loss: 0.4547
Epoch [1/15], Step [72/75], Loss: 0.3878
Epoch [1/15], Step [76/75], Loss: 0.4485
Epoch [2/15], Step [4/75], Loss: 0.4602
Epoch [2/15], Step [8/75], Loss: 0.1593
Epoch [2/15], Step [12/75], Loss: 0.1479
Epoch [2/15], Step [16/75], Loss: 0.0796
Epoch [2/15], Step [20/75], Loss: 0.1462
Epoch [2/15], Step [

In [21]:
# Test model num_epochs = 15
correct = 0

total_test_data = len(newsgroups_test.target)
total_batch = int(total_test_data/batch_size)
for i in range(total_batch + 1):
    batch_x_test,batch_y_test = get_batch(newsgroups_test,i,batch_size)
    articles = Variable(torch.FloatTensor(batch_x_test))
    labels = torch.LongTensor(batch_y_test)
    
    outputs = net(articles)
    _, predicted = torch.max(outputs.data, 1)
   
    correct += (predicted == labels).sum()
    
print('Accuracy of the network on the test articles: %d %%' % (100 * correct / total_test_data))

Accuracy of the network on the test articles: 76 %


In [23]:
num_epochs = 20

In [24]:
# Training num_epochs = 20
# Hàm Loss và Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    #  Lặp lại qua từng gói
    for i in range(total_batch + 1):
        batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()            # Đưa các giá trị gradient về 0
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()                  # Tính toán các gradient  
        optimizer.step()                 # Cập nhật các tham số

        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.data))

Epoch [1/20], Step [4/75], Loss: 3.0007
Epoch [1/20], Step [8/75], Loss: 2.6710
Epoch [1/20], Step [12/75], Loss: 2.2359
Epoch [1/20], Step [16/75], Loss: 1.7174
Epoch [1/20], Step [20/75], Loss: 1.2788
Epoch [1/20], Step [24/75], Loss: 2.1794
Epoch [1/20], Step [28/75], Loss: 1.1663
Epoch [1/20], Step [32/75], Loss: 0.9954
Epoch [1/20], Step [36/75], Loss: 0.9206
Epoch [1/20], Step [40/75], Loss: 0.7143
Epoch [1/20], Step [44/75], Loss: 0.9131
Epoch [1/20], Step [48/75], Loss: 0.7884
Epoch [1/20], Step [52/75], Loss: 0.7827
Epoch [1/20], Step [56/75], Loss: 0.7314
Epoch [1/20], Step [60/75], Loss: 0.5536
Epoch [1/20], Step [64/75], Loss: 0.4110
Epoch [1/20], Step [68/75], Loss: 0.3580
Epoch [1/20], Step [72/75], Loss: 0.6228
Epoch [1/20], Step [76/75], Loss: 0.3584
Epoch [2/20], Step [4/75], Loss: 0.3665
Epoch [2/20], Step [8/75], Loss: 0.2262
Epoch [2/20], Step [12/75], Loss: 0.1920
Epoch [2/20], Step [16/75], Loss: 0.3752
Epoch [2/20], Step [20/75], Loss: 0.1260
Epoch [2/20], Step [

In [25]:
# Test model num_epochs = 20
correct = 0

total_test_data = len(newsgroups_test.target)
total_batch = int(total_test_data/batch_size)
for i in range(total_batch + 1):
    batch_x_test,batch_y_test = get_batch(newsgroups_test,i,batch_size)
    articles = Variable(torch.FloatTensor(batch_x_test))
    labels = torch.LongTensor(batch_y_test)
    
    outputs = net(articles)
    _, predicted = torch.max(outputs.data, 1)
   
    correct += (predicted == labels).sum()
    
print('Accuracy of the network on the test articles: %d %%' % (100 * correct / total_test_data))

Accuracy of the network on the test articles: 79 %
